In [2]:
import pandas as pd
import urllib.request
import numpy as np
from IPython.display import display
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder
from sklearn import metrics

In [3]:
print('Loading case data ...')
cases = pd.read_csv("pancancer_mutations_merged.csv")
print("done.")

Loading case data ...


/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (2,15,17,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


done.


In [8]:
cols = list(cases.columns)
cols[0] = 'Seq'
cases.columns=cols

In [9]:
lmap = pd.Series(cases.cancer_type.values,index=cases.bcr_patient_barcode).to_dict()
vc = pd.pivot_table(cases, index="bcr_patient_barcode", columns="Variant_Classification", values="Seq", aggfunc="count")
vc = vc.fillna(0)
vcdf = pd.DataFrame(vc.to_records())
vcdf['cancer_type'] = vcdf["bcr_patient_barcode"].map(lmap)

In [10]:
vcdf.head()

,bcr_patient_barcode,3'Flank,3'UTR,5'Flank,5'UTR,Frame_Shift_Del,Frame_Shift_Ins,In_Frame_Del,In_Frame_Ins,Intron,Missense_Mutation,Nonsense_Mutation,Nonstop_Mutation,RNA,Silent,Splice_Site,Translation_Start_Site,cancer_type
0,TCGA-02-0003,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,0.0,0.0,1.0,12.0,0.0,0.0,GBM
1,TCGA-02-0033,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0,23.0,2.0,0.0,0.0,10.0,1.0,0.0,GBM
2,TCGA-02-0047,0.0,4.0,0.0,1.0,2.0,0.0,1.0,0.0,2.0,56.0,2.0,1.0,2.0,15.0,1.0,0.0,GBM
3,TCGA-02-0055,1.0,5.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,47.0,1.0,0.0,1.0,16.0,2.0,0.0,GBM
4,TCGA-02-2466,1.0,6.0,0.0,3.0,5.0,1.0,1.0,0.0,5.0,77.0,3.0,0.0,1.0,26.0,13.0,0.0,GBM


In [11]:
data= vcdf.drop(['bcr_patient_barcode', 'cancer_type'], axis=1)
labels=vcdf[['cancer_type']]

In [12]:
shuffle = np.random.permutation(np.arange(data.shape[0]))
X, Y  = data.iloc[shuffle], labels.iloc[shuffle]

In [13]:
train_X = X[:6000]
train_Y = Y[:6000]

test_X = X[6000:]
test_Y = Y[6000:]



In [ ]:
clf = MultinomialNB()
clf = clf.fit(train_X, train_Y.values.ravel())
pred_Y = clf.predict(test_X)
# Model Accuracy, how often is the classifier correct?
print("Accuracy for MNB:",metrics.accuracy_score(test_Y, pred_Y))



In [14]:
clf = LogisticRegression()
clf = clf.fit(train_X, train_Y.values.ravel())
pred_Y = clf.predict(test_X)
# Model Accuracy, how often is the classifier correct?
print("Accuracy for Logistic Regression:",metrics.accuracy_score(test_Y, pred_Y))



/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy for Logistic Regression: 0.12899201596806387


/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [15]:
clf=RandomForestClassifier(n_estimators=50, n_jobs=30)
clf.fit(train_X,train_Y.values.ravel())

pred_Y=clf.predict(test_X)
print("Accuracy for Random Forest Regression:",metrics.accuracy_score(test_Y, pred_Y))

Accuracy for Random Forest Regression: 0.30489021956087825
